In [47]:
%reset -f
import pandas as pd
import requests
import json

In [48]:
# api endpoints
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
player_history_url = 'https://fantasy.premierleague.com/api/element-summary/{player_id}/'

In [49]:
# making request
response = requests.get(url)

# check status
if response.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {response.status_code}.')

The API request was successful.


In [50]:
# pandas settings
pd.options.display.max_rows = 999

In [51]:
# get all players
response = requests.get(url)
players = response.json()['elements']

In [52]:
# mapping positions
element_types = response.json()['element_types']
position_mapping = {}

for pos in element_types:
    position_mapping[pos['id']] = pos['singular_name']

In [53]:
# extract past season player info
player_info = []

print("Please wait a few minutes...")

for player in players:
    player_id = player['id']
    name = player['first_name'] + ' ' + player['second_name']
    position = position_mapping[player['element_type']]
    value = player['now_cost']/10
  
    # get player history
    history_response = requests.get(player_history_url.format(player_id=player_id))
    points = 0

    try:
        history_data = history_response.json()['history_past']
        for season in history_data:
            if season['season_name'] == "2022/23":
                points = season['total_points']
                # points = history_response.json()['history_past'][0]['total_points']
            else: 
                pass
    except (KeyError, IndexError):
        # print(f"Error: No history for {player_id}: {name} found in response")
        points = 0
  
    player_info.append({'name': name,'position': position, 'value': value, 'points': points})

df = pd.DataFrame(player_info)

Please wait a few minutes...


In [57]:
# points per million
df['ppm'] = round(df['points'] / df['value'],2)
df = df.sort_values('ppm', ascending=False)

df

,name,position,value,points,ppm
26,David Raya Martin,Goalkeeper,4.8,166,34.58
327,Bernd Leno,Goalkeeper,4.5,142,31.56
521,Kieran Trippier,Defender,6.5,198,30.46
4,Gabriel dos Santos Magalhães,Defender,4.8,146,30.42
49,Tyrone Mings,Defender,4.3,130,30.23
702,José Malheiro de Sá,Goalkeeper,5.0,148,29.60
354,Alisson Ramses Becker,Goalkeeper,5.5,162,29.45
120,Ben Mee,Defender,4.9,143,29.18
663,Lukasz Fabianski,Goalkeeper,4.4,127,28.86
498,Dan Burn,Defender,4.5,129,28.67


In [58]:
pos_only = df[df['position'] == 'Defender']
pos_only = pos_only.sort_values('points', ascending=False)

pos_only

,name,position,value,points,ppm
521,Kieran Trippier,Defender,6.5,198,30.46
23,Benjamin White,Defender,5.5,156,28.36
353,Trent Alexander-Arnold,Defender,7.9,156,19.75
4,Gabriel dos Santos Magalhães,Defender,4.8,146,30.42
120,Ben Mee,Defender,4.9,143,29.18
518,Fabian Schär,Defender,5.0,139,27.80
49,Tyrone Mings,Defender,4.3,130,30.23
498,Dan Burn,Defender,4.5,129,28.67
496,Sven Botman,Defender,4.5,129,28.67
144,Pervis Estupiñán,Defender,5.3,128,24.15


In [59]:
# checking stats for specific player id
player_id = 118

history_response = requests.get(player_history_url.format(player_id=player_id))
print(json.dumps(history_response.json()['history_past'], indent=2))

[
  {
    "season_name": "2022/23",
    "element_code": 591357,
    "start_cost": 45,
    "end_cost": 44,
    "total_points": 0,
    "minutes": 0,
    "goals_scored": 0,
    "assists": 0,
    "clean_sheets": 0,
    "goals_conceded": 0,
    "own_goals": 0,
    "penalties_saved": 0,
    "penalties_missed": 0,
    "yellow_cards": 0,
    "red_cards": 0,
    "saves": 0,
    "bonus": 0,
    "bps": 0,
    "influence": "0.0",
    "creativity": "0.0",
    "threat": "0.0",
    "ict_index": "0.0",
    "starts": 0,
    "expected_goals": "0.00",
    "expected_assists": "0.00",
    "expected_goal_involvements": "0.00",
    "expected_goals_conceded": "0.00"
  }
]
